# Topic modelling (LDA)

In [95]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint
import nltk
from nltk import word_tokenize
import numpy as np
import gensim
from gensim.models.wrappers import LdaMallet
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pyLDAvis
stops = stopwords.words("english")

In [96]:
columns_to_drop = ['dataFrom', 'date', 'rating', 'extracted_devresp', 'devresp_time']
review_data = pd.read_csv('/Users/antheaang/Downloads/combined_data.csv').drop(columns_to_drop,axis=1)
review_data.head()

,review
0,Great banking app with attractive interest rat...
1,"A bank like no other, no bank have such amazin..."
2,Notice that the drop in interest rate of 0.8% ...
3,Sending money into my GXS account is a breeze ...
4,I have to say that the UI/UX is one of the bes...


In [97]:
review_data.shape

# We have 14K reviews 

(418, 1)

# Data Pre-Processing
### 1. Replacing/Dropping NULL values

In [98]:
review_data.isnull().sum()

# Since our data has no null values will be skip this step

review    0
dtype: int64

### 2. Converting to LOWER case

In [99]:
review_data['clean_review'] = review_data['review'].apply(lambda x: str(x).lower())
review_data.head()

,review,clean_review
0,Great banking app with attractive interest rat...,great banking app with attractive interest rat...
1,"A bank like no other, no bank have such amazin...","a bank like no other, no bank have such amazin..."
2,Notice that the drop in interest rate of 0.8% ...,notice that the drop in interest rate of 0.8% ...
3,Sending money into my GXS account is a breeze ...,sending money into my gxs account is a breeze ...
4,I have to say that the UI/UX is one of the bes...,i have to say that the ui/ux is one of the bes...


### 3. REMOVE NON-ALPHA DATA(DIGITS,PUNCTUATIONS,DIACRITICS)

In [100]:
review_data['clean_review'] = review_data['clean_review'].str.replace(r'[^a-zA-Z\s]', ' ',regex=True) 
review_data.head()

,review,clean_review
0,Great banking app with attractive interest rat...,great banking app with attractive interest rat...
1,"A bank like no other, no bank have such amazin...",a bank like no other no bank have such amazin...
2,Notice that the drop in interest rate of 0.8% ...,notice that the drop in interest rate of ...
3,Sending money into my GXS account is a breeze ...,sending money into my gxs account is a breeze ...
4,I have to say that the UI/UX is one of the bes...,i have to say that the ui ux is one of the bes...


### 4. REMOVING WHITE SPACE

In [101]:
review_data['clean_review'] = review_data['clean_review'].str.replace(r'\s{2,}', ' ',regex=True)
review_data.head()

,review,clean_review
0,Great banking app with attractive interest rat...,great banking app with attractive interest rat...
1,"A bank like no other, no bank have such amazin...",a bank like no other no bank have such amazing...
2,Notice that the drop in interest rate of 0.8% ...,notice that the drop in interest rate of in sa...
3,Sending money into my GXS account is a breeze ...,sending money into my gxs account is a breeze ...
4,I have to say that the UI/UX is one of the bes...,i have to say that the ui ux is one of the bes...


### 5. WORD TOKENIZATION

In [102]:
review_data['clean_review'] = review_data['clean_review'].apply(lambda x: word_tokenize(x))
review_data.head()

,review,clean_review
0,Great banking app with attractive interest rat...,"[great, banking, app, with, attractive, intere..."
1,"A bank like no other, no bank have such amazin...","[a, bank, like, no, other, no, bank, have, suc..."
2,Notice that the drop in interest rate of 0.8% ...,"[notice, that, the, drop, in, interest, rate, ..."
3,Sending money into my GXS account is a breeze ...,"[sending, money, into, my, gxs, account, is, a..."
4,I have to say that the UI/UX is one of the bes...,"[i, have, to, say, that, the, ui, ux, is, one,..."


### 6. REMOVE UNNECESSARY WORDS

In [103]:
review_data['clean_review'] = review_data['clean_review'].apply\
(lambda x:[word for word in x if word not in stopwords.words("english") and len(word) > 3 and word.isalpha()])

review_data.head()

,review,clean_review
0,Great banking app with attractive interest rat...,"[great, banking, attractive, interest, rates, ..."
1,"A bank like no other, no bank have such amazin...","[bank, like, bank, amazing, feature, separate,..."
2,Notice that the drop in interest rate of 0.8% ...,"[notice, drop, interest, rate, saving, account..."
3,Sending money into my GXS account is a breeze ...,"[sending, money, account, breeze, instantaneou..."
4,I have to say that the UI/UX is one of the bes...,"[best, seen, digibank, mobile, finally, strong..."


In [104]:
# for dev_response
'''custom_stopwords = ['Hi', 'please', 'thank you', 'happy', 'thrilled', 'delighted', 'thank', 'there', 'review', 'sorry', 'hope',]
review_data['clean_review'] = review_data['clean_review'].apply\
(lambda x:[word for word in x if word not in custom_stopwords and len(word) > 3 and word.isalpha()]) '''

"custom_stopwords = ['Hi', 'please', 'thank you', 'happy', 'thrilled', 'delighted', 'thank', 'there', 'review', 'sorry', 'hope',]\nreview_data['clean_review'] = review_data['clean_review'].apply(lambda x:[word for word in x if word not in custom_stopwords and len(word) > 3 and word.isalpha()]) "

In [105]:
review_data = review_data[review_data['clean_review'].map(lambda x: len(x)) > 1].reset_index(drop=True)
# Keeping records with more than single words

### 7. LEMMATIZATION

In [106]:
review_data['clean_review'] = review_data['clean_review'].apply\
(lambda x: [WordNetLemmatizer().lemmatize(word) for word in x])
review_data.head()

,review,clean_review
0,Great banking app with attractive interest rat...,"[great, banking, attractive, interest, rate, p..."
1,"A bank like no other, no bank have such amazin...","[bank, like, bank, amazing, feature, separate,..."
2,Notice that the drop in interest rate of 0.8% ...,"[notice, drop, interest, rate, saving, account..."
3,Sending money into my GXS account is a breeze ...,"[sending, money, account, breeze, instantaneou..."
4,I have to say that the UI/UX is one of the bes...,"[best, seen, digibank, mobile, finally, strong..."


### 8. Extracting only NOUN

In [107]:
review_data['clean_review'] = review_data['clean_review'].apply\
(lambda x: [word for word in x if nltk.pos_tag([word])[0][1] == 'NN'])

In [108]:
review_data = review_data[review_data['clean_review'].map(lambda x: len(x)) > 1].reset_index(drop=True)
# Keeping records with more than single words

In [109]:
review_data.head()

,review,clean_review
0,Great banking app with attractive interest rat...,"[banking, interest, rate, please, payee, numbe..."
1,"A bank like no other, no bank have such amazin...","[bank, bank, feature, money, pocket, track, go..."
2,Notice that the drop in interest rate of 0.8% ...,"[notice, drop, interest, rate, account, reason..."
3,Sending money into my GXS account is a breeze ...,"[money, account, breeze, amount, fund, money, ..."
4,I have to say that the UI/UX is one of the bes...,"[digibank, mobile, market, player, focus, cust..."


### 9. Document Term Matrix

In [110]:
dictionary = corpora.Dictionary(review_data['clean_review'])
print(dictionary)

# We have 6724 unique tokens

Dictionary(738 unique tokens: ['account', 'apple', 'banking', 'card', 'debit']...)


In [111]:
doc_term_matrix = review_data['clean_review'].apply(lambda x: dictionary.doc2bow(x))
doc_term_matrix[:10]

# Each tokenized words has been assigned index value and thier count in corpus

0    [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1...
1    [(5, 3), (9, 1), (10, 3), (11, 1), (12, 1), (1...
2    [(0, 9), (5, 7), (9, 6), (10, 1), (34, 1), (35...
3    [(0, 2), (2, 1), (8, 1), (23, 2), (30, 1), (43...
4    [(2, 1), (8, 1), (10, 1), (26, 1), (33, 1), (6...
5    [(0, 1), (10, 4), (42, 1), (61, 1), (83, 1), (...
6    [(2, 1), (5, 1), (9, 1), (10, 1), (32, 1), (74...
7    [(2, 1), (7, 1), (10, 3), (17, 1), (23, 1), (3...
8    [(17, 1), (25, 1), (32, 1), (33, 1), (74, 1), ...
9    [(0, 2), (6, 1), (8, 1), (10, 3), (21, 1), (56...
Name: clean_review, dtype: object

### SEEDING LDA

In [112]:
App_Responsiveness = ['freeze', 'crash', 'buggy', 'speed', 'fast', 'crash', 'swift', 'rapid', 'seamless', 'smooth', 'wait', 'snappy', 'load', 'efficient', 'error']
Money_Growth = ['drop', 'interest rate', 'ROI', 'interest', 'rates', 'savings', 'p.a.']
Customer_Services = ['wait time', 'efficient', 'response', 'customer', 'support', 'hotline', 'email', 'feedback','suggestion']
Services_and_Products = ['deposit', 'repayment', 'loan', 'pocket', 'digital', 'transfer', 'payee', 'saving', 'convenient', 'QR', 'deposit']
User_Interface = ['ease', 'use', 'easy', 'clear', 'friendly', 'seamless', 'functionality', 'graphics', 'design', 'UI', 'UX', 'theme', 'UI/UX', 'interface', 'intuitive']
Credit_card = ['rebates', 'debit', 'creditcard', 'credit', 'card']
Login_Account_Setup = ['registration', 'singpass', 'password', 'biometric', 'onboard', 'setup', 'set', 'up', 'login' , 'open', 'signup', 'fingerprint', 'launch', 'register', 'create', 'application', '2FA', 'download', 'biometrics', 'sign']
Competition = ['Trust', 'maribank', 'T' , 'bills', 'OCBC', 'Timo']
Safety = ['password', 'pin', 'particulars', 'hack', 'security', 'secure', 'safe']
Customer_trust = ['misleading', 'deceive', 'fake' , 'false', 'scam']

In [113]:
seed_topics = {}
for word in App_Responsiveness:
    seed_topics[word] = 0
for word in Money_Growth:
    seed_topics[word] = 1
for word in Customer_Services:
    seed_topics[word] = 2
for word in Services_and_Products:
    seed_topics[word] = 3
for word in User_Interface:
    seed_topics[word] = 4
for word in Credit_card:
    seed_topics[word] = 5
for word in Login_Account_Setup:
    seed_topics[word] = 6
for word in Competition:
    seed_topics[word] = 7
for word in Safety:
    seed_topics[word] = 8
for word in Customer_trust:
    seed_topics[word] = 9

In [114]:
def create_eta(priors, etadict, ntopics):
    eta = np.full(shape=(ntopics, len(etadict)), fill_value=1) # create a (ntopics, nterms) matrix and fill with 1
    for word, topic in priors.items(): # for each word in the list of priors
        keyindex = [index for index,term in etadict.items() if term==word] # look up the word in the dictionary
        if (len(keyindex)>0): # if it's in the dictionary
            eta[topic,keyindex[0]] = 1e7  # put a large number in there
    eta = np.divide(eta, eta.sum(axis=0)) # normalize so that the probabilities sum to 1 over all topics
    return eta

In [115]:
eta = create_eta(seed_topics, dictionary, 10)

### LDA

In [116]:
os.environ['MALLET_HOME'] = "/Users/antheaang/Downloads/mallet-2.0.8"
print(os.getenv("MALLET_HOME"))
mallet_path = "/Users/antheaang/Downloads/mallet-2.0.8/bin/mallet"

ldamallet = LdaMallet(mallet_path, corpus=doc_term_matrix, num_topics=10, id2word=dictionary)
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=review_data['clean_review'], dictionary=dictionary, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

/Users/antheaang/Downloads/mallet-2.0.8


Mallet LDA: 10 topics, 4 topic bits, 1111 topic mask
Data loaded.
max tokens: 49
total tokens: 2292
<10> LL/token: -7.91195
<20> LL/token: -7.65623
<30> LL/token: -7.53719
<40> LL/token: -7.41482

0	5	interest rate simple trust waste point till product news drop consumer explain cool chat attempt effort beautiful scammer industry spent 
1	5	sign user experience feature deposit design money message opening developer lousy suggestion start provide market ecosystem signing meet proper build 
2	5	account bank payee interest increase register reason year dont grow term spend response asap maximum invitation space public pas gxbank 
3	5	issue fast singpass review update hope page password min joke setup criterion buggy display yesterday loan response cash doubt kudos 
4	5	phone money wait transfer screen service customer ease star kind join advice le information clean company buddy sept emergency type 
5	5	login banking apps digital super rating problem process store day change support attem

[(0,
  [('interest', 0.3146551724137931),
   ('rate', 0.23706896551724138),
   ('increase', 0.03879310344827586),
   ('point', 0.03017241379310345),
   ('consumer', 0.017241379310344827),
   ('lack', 0.017241379310344827),
   ('thing', 0.017241379310344827),
   ('drop', 0.017241379310344827),
   ('device', 0.01293103448275862),
   ('bait', 0.01293103448275862)]),
 (1,
  [('user', 0.12322274881516587),
   ('offer', 0.052132701421800945),
   ('detail', 0.04265402843601896),
   ('super', 0.037914691943127965),
   ('design', 0.037914691943127965),
   ('security', 0.02843601895734597),
   ('support', 0.02843601895734597),
   ('year', 0.02843601895734597),
   ('payment', 0.02843601895734597),
   ('click', 0.023696682464454975)]),
 (2,
  [('account', 0.23412698412698413),
   ('login', 0.08333333333333333),
   ('fund', 0.06349206349206349),
   ('application', 0.051587301587301584),
   ('told', 0.027777777777777776),
   ('page', 0.023809523809523808),
   ('star', 0.01984126984126984),
   ('proc


0	5	interest rate increase point lack thing drop consumer straight bait device switch notice case everyday kudos pull alternative select mistake 
1	5	user offer detail super design payment security year click flexiloan support message news min developer display buggy hassle grow thing 
2	5	account login fund application page told proceed star till useless repayment quick advice public borrow bother meet register project popped 
3	5	loan phone fast issue update review screen amount show response ease smooth day code limit setup earn approval clean load 
4	5	sign download rating problem reason attempt store week navigate simple touch play hope information respond seamless theme ocbc product force 
5	5	pocket wait experience digital love deposit function space create write joke feature criterion singtel told question proper competitor start standard 
6	5	banking apps grab error month card dont hope debit registration info link register today effort past explain cool le cash 
7	5	transfer


Coherence Score:  0.6171789850040147


### Visualizing LDA model topics

In [117]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamallet,doc_term_matrix,dictionary)
vis

AttributeError: module 'pyLDAvis' has no attribute 'gensim'

In [118]:
ldamallet.print_topics()

[(0,
  '0.315*"interest" + 0.237*"rate" + 0.039*"increase" + 0.030*"point" + 0.017*"consumer" + 0.017*"lack" + 0.017*"thing" + 0.017*"drop" + 0.013*"device" + 0.013*"bait"'),
 (1,
  '0.123*"user" + 0.052*"offer" + 0.043*"detail" + 0.038*"super" + 0.038*"design" + 0.028*"security" + 0.028*"support" + 0.028*"year" + 0.028*"payment" + 0.024*"click"'),
 (2,
  '0.234*"account" + 0.083*"login" + 0.063*"fund" + 0.052*"application" + 0.028*"told" + 0.024*"page" + 0.020*"star" + 0.020*"proceed" + 0.020*"repayment" + 0.016*"register"'),
 (3,
  '0.103*"loan" + 0.099*"phone" + 0.075*"fast" + 0.071*"issue" + 0.056*"update" + 0.056*"review" + 0.040*"screen" + 0.032*"amount" + 0.024*"show" + 0.024*"response"'),
 (4,
  '0.201*"sign" + 0.057*"download" + 0.038*"rating" + 0.038*"problem" + 0.033*"reason" + 0.029*"simple" + 0.029*"store" + 0.029*"attempt" + 0.024*"navigate" + 0.024*"hope"'),
 (5,
  '0.161*"pocket" + 0.085*"wait" + 0.081*"experience" + 0.058*"digital" + 0.049*"love" + 0.036*"deposit" + 0.

### Creating a lookup table for topics

In [119]:
topic_lookup_data = pd.DataFrame((ldamallet.print_topics()),columns=['Topic_Number','Top_Keywords'])


topic_lookup_data['Topic_Name'] = ['App Responsiveness','Money Growth (Interest Rates)','Customer Services','Services & Products','User Interface','Credit card', 'Login & Account Setup', 'Competition', 'Safety', 'Customer trust']


topic_lookup_data = topic_lookup_data[['Topic_Number','Topic_Name','Top_Keywords']]


topic_lookup_data['Top_Keywords'] = topic_lookup_data.Top_Keywords.str\
.replace(r'[^a-z]',' ',regex=True).apply(lambda x: x.split())


topic_lookup_data.style.set_properties(subset=['Top_Keywords'], **{'width': '300px'})

,Topic_Number,Topic_Name,Top_Keywords
0,0,App Responsiveness,"['interest', 'rate', 'increase', 'point', 'consumer', 'lack', 'thing', 'drop', 'device', 'bait']"
1,1,Money Growth (Interest Rates),"['user', 'offer', 'detail', 'super', 'design', 'security', 'support', 'year', 'payment', 'click']"
2,2,Customer Services,"['account', 'login', 'fund', 'application', 'told', 'page', 'star', 'proceed', 'repayment', 'register']"
3,3,Services & Products,"['loan', 'phone', 'fast', 'issue', 'update', 'review', 'screen', 'amount', 'show', 'response']"
4,4,User Interface,"['sign', 'download', 'rating', 'problem', 'reason', 'simple', 'store', 'attempt', 'navigate', 'hope']"
5,5,Credit card,"['pocket', 'wait', 'experience', 'digital', 'love', 'deposit', 'function', 'create', 'space', 'write']"
6,6,Login & Account Setup,"['banking', 'apps', 'grab', 'error', 'month', 'card', 'singpass', 'debit', 'dont', 'link']"
7,7,Competition,"['transfer', 'feature', 'payee', 'number', 'interface', 'everytime', 'password', 'force', 'feel', 'credit']"
8,8,Safety,"['time', 'money', 'singpass', 'email', 'waste', 'multiple', 'change', 'access', 'transaction', 'list']"
9,9,Customer trust,"['bank', 'account', 'work', 'trust', 'launch', 'simple', 'process', 'kind', 'singapore', 'input']"


### Creating new columns and inserting topic numbers and names

In [120]:
for index,sent in enumerate(ldamallet[doc_term_matrix]):
  topic_num =[]
  topic_details = sorted(sent,key=lambda x: x[1], reverse=True)[:2] # Getting top 2 topics in descending order
  topic_num.append(topic_details[0][0]) # Appending top topic
  if len(topic_details) > 1:
    if topic_details[1][1] > 0.35: # Appending second topic only if it has more than 35% influence on current row
      topic_num.append(topic_details[1][0])
  review_data.loc[index,'Topic_Number'] = ','.join(str(x) for x in sorted(topic_num))

  
review_data.head()

,review,clean_review,Topic_Number
0,Great banking app with attractive interest rat...,"[banking, interest, rate, please, payee, numbe...",7
1,"A bank like no other, no bank have such amazin...","[bank, bank, feature, money, pocket, track, go...",9
2,Notice that the drop in interest rate of 0.8% ...,"[notice, drop, interest, rate, account, reason...",0
3,Sending money into my GXS account is a breeze ...,"[money, account, breeze, amount, fund, money, ...",8
4,I have to say that the UI/UX is one of the bes...,"[digibank, mobile, market, player, focus, cust...",5


In [121]:
def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamallet, corpus=doc_term_matrix, texts=review_data['clean_review'])

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,7.0,0.1352,"transfer, feature, payee, number, interface, e...","[banking, interest, rate, please, payee, numbe..."
1,1,9.0,0.1308,"bank, account, work, trust, launch, simple, pr...","[bank, bank, feature, money, pocket, track, go..."
2,2,0.0,0.3367,"interest, rate, increase, point, consumer, lac...","[notice, drop, interest, rate, account, reason..."
3,3,8.0,0.1787,"time, money, singpass, email, waste, multiple,...","[money, account, breeze, amount, fund, money, ..."
4,4,5.0,0.1231,"pocket, wait, experience, digital, love, depos...","[digibank, mobile, market, player, focus, cust..."
5,5,9.0,0.2418,"bank, account, work, trust, launch, simple, pr...","[slot, account, month, nothing, whenever, anyo..."
6,6,9.0,0.1443,"bank, account, work, trust, launch, simple, pr...","[awesome, design, theme, banking, institution,..."
7,7,3.0,0.1573,"loan, phone, fast, issue, update, review, scre...","[fast, reversal, money, bank, time, life, fast..."
8,8,5.0,0.1267,"pocket, wait, experience, digital, love, depos...","[user, design, navigate, pocket, feature, wait..."
9,9,9.0,0.2145,"bank, account, work, trust, launch, simple, pr...","[joke, nothing, input, phone, number, nothing,..."


In [122]:
for index,topic_num in enumerate(review_data.Topic_Number):
  topic_name_list=[]
  for single_topic_num in topic_num.split(','):
    single_topic_num=int(single_topic_num)
    topic_name_list.append(topic_lookup_data.loc\
                           [topic_lookup_data.Topic_Number == single_topic_num,'Topic_Name'][single_topic_num]) 
  # Extracting topic names from lookup table
  review_data.loc[index,'Topic_Name'] =' & '.join(topic_name_list)

In [123]:
review_data

,review,clean_review,Topic_Number,Topic_Name
0,Great banking app with attractive interest rat...,"[banking, interest, rate, please, payee, numbe...",7,Competition
1,"A bank like no other, no bank have such amazin...","[bank, bank, feature, money, pocket, track, go...",9,Customer trust
2,Notice that the drop in interest rate of 0.8% ...,"[notice, drop, interest, rate, account, reason...",0,App Responsiveness
3,Sending money into my GXS account is a breeze ...,"[money, account, breeze, amount, fund, money, ...",8,Safety
4,I have to say that the UI/UX is one of the bes...,"[digibank, mobile, market, player, focus, cust...",5,Credit card
...,...,...,...,...
312,digital bank app does not work when it already...,"[digital, bank, work]",9,Customer trust
313,Not ready to roll out completely. Aint even al...,"[roll, aint, create, account, invitation]",9,Customer trust
314,"Can not download yet, just always show pending...","[download, show]",3,Services & Products
315,Looks cool and sleek! Can I get an invite if I...,"[look, cool, sleek, invite, star, review]",2,Customer Services


In [124]:
review_data.to_csv('topics_review.csv')
